In [1]:
input_file = "dream.txt"

import json
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import tensorflow_probability as tfp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.stats import gaussian_kde

# TODO: Change root path
root = "/Users/jay/Desktop/Bachelorarbeit"

import sys

sys.path.append(f"{root}/Implementation")
from dependencies.hbv_sask.model import HBVSASKModel as hbvmodel
from src.execute_model import run_model_single_parameter_node
from src.construct_model import get_model


ndims = 7
dims = ["TT", "C0", "beta", "ETF", "FC", "FRAC", "K2"]
colors = sns.color_palette(n_colors=ndims)

testConfigPath = f"{root}/test_config.json"
with open(testConfigPath, "r") as file:
    run_config = json.load(file)

configPath = run_config["configPath"]
basis = run_config["basis"]
model = get_model(configPath, basis)

start_date: 2004-01-01 00:00:00
start_date_predictions: 2005-01-01 00:00:00
end_date: 2006-01-01 00:00:00
simulation length: 365
full_data_range is 732 hours including spin_up_length of 366 hours
simulation_range is of length 366 hours


In [2]:
# Construct params
configurationObject = model.configurationObject
param_lower = []
param_upper = []
for param in configurationObject["parameters"]:
    if param["distribution"] == "Uniform":
        param_lower.append(param["lower"])
        param_upper.append(param["upper"])
    else:
        raise NotImplementedError(
            f"Sorry, the distribution {param['distribution']} is not supported yet"
        )
param_lower = np.array(param_lower)
param_upper = np.array(param_upper)

In [3]:
samples = pd.read_csv(f"{input_file}")
samples

,TT,C0,beta,ETF,FC,FRAC,K2
0,0.911258,1.356576,2.132870,0.235092,291.572193,0.215022,0.043257
1,0.911258,1.356576,2.132870,0.235092,291.572193,0.215022,0.043257
2,-0.233729,0.588043,2.198162,0.113685,233.553188,0.328521,0.043458
3,-0.233729,0.588043,2.198162,0.113685,233.553188,0.328521,0.043458
4,-0.233729,0.588043,2.198162,0.113685,233.553188,0.328521,0.043458
...,...,...,...,...,...,...,...
884,1.951661,6.653201,1.385673,0.099440,209.911936,0.171404,0.025641
885,2.792069,5.376126,1.751294,0.129530,115.271779,0.137601,0.041602
886,3.831516,5.807979,2.020655,0.008888,157.787768,0.276033,0.042169
887,3.831516,5.807979,2.020655,0.008888,157.787768,0.276033,0.042169


# Plotting

In [5]:
fig = make_subplots(rows=3, cols=3)

# Histogram and KDE
for i, col in enumerate(samples.columns):
    row = (i // 3) + 1
    col_idx = (i % 3) + 1

    # Histogram
    fig.add_trace(
        go.Histogram(x=samples[col], name=col, histnorm="probability density"),
        row=row,
        col=col_idx,
    )

    # KDE calculation
    kde = gaussian_kde(samples[col])
    x_values = np.linspace(samples[col].min(), samples[col].max(), 300)
    kde_values = kde(x_values)
    fig.add_trace(
        go.Scatter(
            x=x_values, y=kde_values, mode="lines", name=f"KDE {col}", showlegend=False
        ),
        row=row,
        col=col_idx,
    )

# Update layout
fig.update_layout(
    height=700,
    width=1000,
    title_text="DREAM: Parameters Overview",
)
fig.show()